# Trabajo con Avro y Hive

In [1]:
! mkdir -p avro-hive

In [2]:
import os
os.chdir("avro-hive")

In [3]:
! pwd

/media/notebooks/avro-hive


## Deja los ficheros de datos en la ruta /media/notebooks

In [4]:
! hadoop fs -mkdir datoshive
! hadoop fs -put ../part-00000 datoshive
! hadoop fs -put ../nuevoregistro.avro datoshive
! hadoop fs -put ../nuevoregistro2.avro datoshive
! hadoop fs -put ../000000_0 datoshive


put: `datoshive/part-00000': File exists


## Creamos tabla interna

Vamos a crear una tabla interna basada en Avro y la poblaremos con registros de un fichero de datos avro existente.

In [5]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "drop table avro_datacleaninternal;"
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "drop table avroB_datacleaninternal;"
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "drop table avrodatacleanexternal;"

2019-05-14 09:44:56,090 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514094444_6e8748dd-f5a9-423e-b804-95f908c7c3d0): drop table avro_datacleaninternal
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20190514094444_6e8748dd-f5a9-423e-b804-95f908c7c3d0); Time taken: 0.571 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514094444_6e8748dd-f5a9-423e-b804-95f908c7c3d0): drop table avro_datacleaninternal
INFO  : Starting task [Stage-0:DDL] in serial mode

In [6]:
%%writefile avrohive.hql

create  table avro_datacleaninternal (
  idref  STRING,   
  ident  STRING,
  gsm19023 FLOAT,  
  gsd19024 FLOAT, 
  gsd19025 FLOAT,    
  gsd19026 FLOAT, 
  genetitle STRING,   
  genesymbol STRING,
  geneID SMALLINT,    
  uniGenetitle STRING,
  uniGenesymbol STRING,   
  uniGeneID STRING,
  NucleotideTitle STRING) 
STORED AS AVRO;

Writing avrohive.hql


In [7]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -f avrohive.hql

2019-05-14 09:45:46,134 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/bioinformatic> 
nal (bc:hive2://localhost:10000/bioinformatic> create  table avro_datacleaninter 
0: jdbc:hive2://localhost:10000/bioinformatic>   idref  STRING,   
0: jdbc:hive2://localhost:10000/bioinformatic>   ident  STRING,
0: jdbc:hive2://localhost:10000/bioinformatic>   gsm19023 FLOAT,  
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19024 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19025 FLOAT,    
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19026 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   genetitle STRING,   
0: jdbc:hi

In [14]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "LOAD DATA INPATH '/user/root/datoshive/000000_0' INTO TABLE avro_datacleaninternal;" 

2019-05-14 09:49:30,695 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514094949_6d7ca608-38d8-4f03-a53f-8f0be30304e6): LOAD DATA INPATH '/user/root/datoshive/000000_0' INTO TABLE avro_datacleaninternal
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20190514094949_6d7ca608-38d8-4f03-a53f-8f0be30304e6); Time taken: 0.087 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514094949_6d7ca608-38d8-4f03-a53f-8f0be30304e6): LOAD DATA INPATH '/user/root/datosh

Veremos que los datos se han introducido en la tabla interna basada en avro, y que al principio del fichero de datos aparece el esquema de avro.

In [20]:
! hadoop fs -cat /user/cloudera/bioinformatica/avro_datacleaninternal/000000_0 | head

{"type":"record","name":"avro_datacleaninternal","namespace":"bioinformatica","fields":[{"name":"idref","type":["null","string"],"default":null},{"name":"ident","type":["null","string"],"default":null},{"name":"gsm19023","type":["null","float"],"default":null},{"name":"gsd19024","type":["null","float"],"default":null},{"name":"gsd19025","type":["null","float"],"default":null},{"name":"gsd19026","type":["null","float"],"default":null},{"name":"genetitle","type":["null","string"],"default":null},{"name":"genesymbol","type":["null","string"],"default":null},{"name":"geneid","type":["null","int"],"default":null},{"name":"unigenetitle","type":["null","string"],"default":null},{"name":"unigenesymbol","type":["null","string"],"default":null},{"name":"unigeneid","type":["null","string"],"default":null},{"name":"nucleotidetitle","type":["null","string"],"default":null}]} ��=��3c���nic���100001_atCd3g�5�E3C�E��9D�9D<CD3 antigen, gamma polypeptideCd3g��   NMouse CD3-gamma (

In [21]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "select count(*) from avro_datacleaninternal where genetitle like '%human%';" 

2019-05-14 09:59:21,243 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514095959_a0e6e08d-1205-4424-b5ad-c116d9ee7b4f): select count(*) from avro_datacleaninternal where genetitle like '%human%'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:_c0, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514095959_a0e6e08d-1205-4424-b5ad-c116d9ee7b4f); Time taken: 0.659 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514095959_a0e6e08d-1205-4424-b5ad-c116d9ee7b4f

## Creamos tabla externa

In [33]:
! hadoop fs -mkdir   /user/cloudera/datosavrohive

In [34]:
! hadoop fs -put ../000000_0  /user/cloudera/datosavrohive

In [36]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "drop table avrodatacleanexternal;"

2019-05-14 10:30:56,419 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514103030_8adb8af4-4d4f-4a0c-a7ea-e9a8e4374e53): drop table avrodatacleanexternal
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20190514103030_8adb8af4-4d4f-4a0c-a7ea-e9a8e4374e53); Time taken: 0.019 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514103030_8adb8af4-4d4f-4a0c-a7ea-e9a8e4374e53): drop table avrodatacleanexternal
INFO  : Starting task [Stage-0:DDL] in serial mode
I

In [37]:
%%writefile ejerciciohive.hql

create external table avrodatacleanexternal (
  idref  STRING,   
  ident  STRING, 
  gsm19023 FLOAT,  
  gsd19024 FLOAT, 
  gsd19025 FLOAT,   
  gsd19026 FLOAT, 
  genetitle STRING,   
  genesymbol STRING,
  geneID SMALLINT,   
  uniGenetitle STRING,
  uniGenesymbol STRING,  
  uniGeneID STRING,
  NucleotideTitle STRING) 
STORED AS AVRO
LOCATION '/user/cloudera/datosavrohive';



Overwriting ejerciciohive.hql


In [38]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -f ejerciciohive.hql

2019-05-14 10:31:47,863 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/bioinformatic> 
anexternal (2://localhost:10000/bioinformatic> create external table avrodatacle 
0: jdbc:hive2://localhost:10000/bioinformatic>   idref  STRING,   
0: jdbc:hive2://localhost:10000/bioinformatic>   ident  STRING, 
0: jdbc:hive2://localhost:10000/bioinformatic>   gsm19023 FLOAT,  
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19024 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19025 FLOAT,   
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19026 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   genetitle STRING,   
0: jdbc:hi

In [40]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "DESCRIBE FORMATTED avrodatacleanexternal;"

2019-05-14 10:34:30,337 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514103434_a21a3f82-47ec-4068-a8e4-c692f5821583): DESCRIBE FORMATTED avrodatacleanexternal
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:col_name, type:string, comment:from deserializer), FieldSchema(name:data_type, type:string, comment:from deserializer), FieldSchema(name:comment, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514103434_a21a3f82-47ec-4068-a8e4-c692f5821583); Time taken: 0.076 seconds
INFO  : Concurrency mode is disabled

## Modificando el esquema Avro

Vamos a modificar la tabla basada en Avro añadiendo una nueva columna. Veremos que para los datos existentes esta columna adquiere valor NULL. Luego insertaremos registros que siguen el nuevo esquema, y veremos que las consultas devuelven los datos de esta columna adecuados (null para los items insertados antes del cambio de esquema, y el valor para los datos insertados tras el cambio).

Tras el cambio, añadimos un registro que sigue el esquema viejo, sin problemas.

In [41]:
%%writefile avrohive.hql

ALTER TABLE avrodatacleanexternal ADD COLUMNS (mynewcolumn STRING);

Overwriting avrohive.hql


In [42]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -f avrohive.hql

2019-05-14 10:36:51,614 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/bioinformatic> 
 ADD COLUMNS (mynewcolumn STRING);oinformatic> ALTER TABLE avrodatacleanexternal 
INFO  : Compiling command(queryId=hive_20190514103636_a79a999a-36b9-4654-a564-a92751ac8346): ALTER TABLE avrodatacleanexternal ADD COLUMNS (mynewcolumn STRING)
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20190514103636_a79a999a-36b9-4654-a564-a92751ac8346); Time taken: 0.049 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Execut

In [43]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "describe FORMATTED avrodatacleanexternal";

2019-05-14 10:37:01,982 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514103737_206e98e1-e8af-4035-939d-facf4013a62c): describe FORMATTED avrodatacleanexternal
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:col_name, type:string, comment:from deserializer), FieldSchema(name:data_type, type:string, comment:from deserializer), FieldSchema(name:comment, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514103737_206e98e1-e8af-4035-939d-facf4013a62c); Time taken: 0.09 seconds
INFO  : Concurrency mode is disabled,

In [44]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%';" 

2019-05-14 10:39:32,696 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514103939_f9896c81-1818-419d-988f-a019b19393a1): select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:mynewcolumn, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514103939_f9896c81-1818-419d-988f-a019b19393a1); Time taken: 0.115 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514103939_f9896c81-1818-419d-988f-a019

In [46]:
! hadoop fs -cp datoshive/nuevoregistro.avro /user/cloudera/datosavrohive

In [47]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%';" 

2019-05-14 10:41:20,542 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514104141_ae1e64d6-ec35-4acb-91e0-adc88f15c4f2): select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:mynewcolumn, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514104141_ae1e64d6-ec35-4acb-91e0-adc88f15c4f2); Time taken: 0.097 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514104141_ae1e64d6-ec35-4acb-91e0-adc8

Añadimos un nuevo registro que sigue el esquema original sin la nueva columna.

In [48]:
! hadoop fs -cp datoshive/nuevoregistro2.avro /user/cloudera/datosavrohive

In [49]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -e "select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%';" 

2019-05-14 10:43:04,942 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 4ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=hive_20190514104343_b6e49a83-c569-4796-a269-5d0371b2f005): select mynewcolumn from avrodatacleanexternal where genetitle like '%CD3%'
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:mynewcolumn, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20190514104343_b6e49a83-c569-4796-a269-5d0371b2f005); Time taken: 0.082 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20190514104343_b6e49a83-c569-4796-a269-5d03

## Herramientas para trabajar con Avro

In [51]:
! java -jar /usr/lib/avro/avro-tools-1.7.6-cdh5.7.0.jar

Version 1.7.6-cdh5.7.0 of Apache Avro
Copyright 2010 The Apache Software Foundation

This product includes software developed at
The Apache Software Foundation (http://www.apache.org/).

C JSON parsing provided by Jansson and
written by Petri Lehtinen. The original software is
available from http://www.digip.org/jansson/.
----------------
Available tools:
          cat  extracts samples from files
      compile  Generates Java code for the given schema.
       concat  Concatenates avro files without re-compressing.
   fragtojson  Renders a binary-encoded Avro datum as JSON.
     fromjson  Reads JSON records and writes an Avro data file.
     fromtext  Imports a text file into an avro data file.
      getmeta  Prints out the metadata of an Avro data file.
    getschema  Prints out schema of an Avro data file.
          idl  Generates a JSON schema from an Avro IDL file
 idl2schemata  Extract JSON schemata of the types from an Avro IDL file
       induce  Induce schema/protocol from Java

## Ejercicio propuesto: Insertar datos de una tabla existente. 

Ahora vamos a repetir los mismos pasos pero ahora insertaremos en la tabla basada en avro los datos de una de las tablas existentes. Se trata de la tabla datacleaninternal, que no está codificada como avro. Comprobar que la cuenta de elementos de la tabla original (desde la que exportamos datos a avro) es la misma que en la tabla avro recién creada.

In [52]:
%%writefile avrohive.hql

create  table avroB_datacleaninternal (
  idref  STRING,   
  ident  STRING,
  gsm19023 FLOAT,  
  gsd19024 FLOAT, 
  gsd19025 FLOAT,    
  gsd19026 FLOAT, 
  genetitle STRING,   
  genesymbol STRING,
  geneID SMALLINT,    
  uniGenetitle STRING,
  uniGenesymbol STRING,   
  uniGeneID STRING,
  NucleotideTitle STRING) 
STORED AS AVRO;

Overwriting avrohive.hql


In [53]:
! beeline -u "jdbc:hive2://localhost:10000/bioinformatica" -f avrohive.hql

2019-05-14 10:47:05,522 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/bioinformatica
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/bioinformatic> 
rnal (c:hive2://localhost:10000/bioinformatic> create  table avroB_datacleaninte 
0: jdbc:hive2://localhost:10000/bioinformatic>   idref  STRING,   
0: jdbc:hive2://localhost:10000/bioinformatic>   ident  STRING,
0: jdbc:hive2://localhost:10000/bioinformatic>   gsm19023 FLOAT,  
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19024 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19025 FLOAT,    
0: jdbc:hive2://localhost:10000/bioinformatic>   gsd19026 FLOAT, 
0: jdbc:hive2://localhost:10000/bioinformatic>   genetitle STRING,   
0: jdbc:hi

In [1]:
# Orden para insertar en la tabla creada arriba los registros de  
# la tabla datacleaninternal, que ya existe en Hive.